<a href="https://colab.research.google.com/github/Oleonn/DataMining/blob/Classification/CNN_Classification_automatique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#-----Imports et installations-----
#Pour se connecter au drive et acceder a ses fichiers
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#Pour l'apprentissage profond
import os
from fastai.vision.all import *
from fastai.vision import *
import pandas as pd

In [41]:
Espece_visee = "Solcan"
Chemin_donnees = "/content/drive/MyDrive/Projet_mellifere/Solidago_canadensis"

In [15]:
Noms_classes = []
for Noms_dossiers in os.listdir(Chemin_donnees):
    if Noms_dossiers.endswith(".txt"):
        continue
    else:
        Noms_classes.append(Noms_dossiers)

In [30]:
Chemins_images = []
Etiquettes = []

for i in range(len(Noms_classes)):
    Classe_nom = Noms_classes[i]
    if "." in Classe_nom:
       continue
    else:        
        Noms_images = os.listdir(f"{Chemin_donnees}/{Classe_nom}")
        for Image_nom in Noms_images:
            Image_chemin = f"{Chemin_donnees}/{Classe_nom}/{Image_nom}"
            Chemins_images.append(Image_chemin)
            Etiquettes.append(i)

In [31]:
df = pd.DataFrame(
    {
        "Nom": Chemins_images,
        "Etiquette": Etiquettes
    }
)

In [32]:
df.to_csv(
    path_or_buf=f"{Chemin_donnees}/etiquettes.csv",
    index=False
)

Telechargement des donnees

In [33]:
dls = ImageDataLoaders.from_folder(
      path=Chemin_donnees,
      item_tfms=Resize(224),
      bs=16,
      batch_tfms=[Normalize.from_stats(*imagenet_stats), RandTransform()],
      valid_pct=0.2
)

Creation du modele

In [ ]:
Modele = vision_learner(
    dls=dls,
    arch=models.resnet18,
    metrics=[accuracy, error_rate]
)

In [40]:
Nb_epochs = 1
Performance = []

for epoch in range(Nb_epochs):
    Modele.fit_one_cycle(1)
    Modele.save(f"CNN_{Espece_visee}_epoch{epoch}.pth")
    Perfo = Modele.recorder.final_record
    Performance.append(Perfo)

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,1.198279,1.065572,0.770000,0.230000,03:56
